# Deploy Document Classification Custom Skill

This tutorial shows how to deploy a document classification custom skill for Cognitive Search. We will use the document classifier that was created by *01_Train_AML_Model.ipynb*. If you have not already, please run that script.

For more information on using custom skills with Cognitive Search, please see this [page](https://docs.microsoft.com/en-us/azure/search/cognitive-search-custom-skill-interface)

### 1.0 Import Packages

In [ ]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License.

import numpy as np

import azureml
from azureml.core import Workspace, Run

# display the core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

### 2.0 Connect to Workspace
Create a workspace object. If you already have a workspace and a config.json file you can use `ws = Workspace.from_config()` instead.

In [ ]:
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.get(name = "", resource_group = "", subscription_id = "")
print(ws.name, ws.location, ws.resource_group, sep = '\t')

### 3.0 Register Model
The last step in the training script wrote the file outputs/sklearn_mnist_model.pkl in a directory named outputs.

Register the model in the workspace so that you (or other collaborators) can query, examine, and deploy this model.

In [ ]:
model = Model.register(model_path="outputs/newsgroup_classifier.pkl",
                        model_name="newsgroup_classifier",
                        tags={"data": "newsgroup", "document": "classification"},
                        description="document classifier for newsgroup20",
                        workspace=ws)

print(model.id)

### 4.0 Create Scoring Script
Create the scoring script, called score.py, used by the web service call to show how to use the model.

You must include two required functions into the scoring script:
- The init() function, which typically loads the model into a global object. This function is run only once when the Docker container is started.
- The run(input_data) function uses the model to predict a value based on the input data. Inputs and outputs to the run typically use JSON for serialization and de-serialization, but other formats are supported.

*The **run function** has been specifically tailored to deploy the model as a custom skill. This means that inputs & outputs are formatted correctly and any errors will be returned in a format usable by Cognitive Search*.

In [ ]:
%%writefile score.py
import json
import numpy as np
import pandas as pd
import os
import pickle
from sklearn.externals import joblib

from azureml.core.model import Model

def init():
    global model
    # retreive the path to the model file using the model name
    model_path = Model.get_model_path(model_name='newsgroup_classifier')
    model = joblib.load(model_path)
    
def convert_to_df(my_dict):
    df = pd.DataFrame(my_dict)
    data = df['data'].tolist()
    index = df['recordId'].tolist()
    return pd.DataFrame(data, index = index)

def run(raw_data):
    data = json.loads(raw_data)
    # Converting the input dictionary to a dataframe
    try:
        df = convert_to_df(data)
    # Returning error message for each item in batch if data not in correct format 
    except:
        df = pd.DataFrame(data)
        index = df['recordId'].tolist()
        message = "Request for batch is not in correct format"
        output_list = [{'recordId': i, 'data': {}, "errors": [{'message': message}]} for i in index]
        return {'values': output_list}
    
    output_list = []
    for index, row in df.iterrows():
        output = {'recordId': index, 'data': {}}
        try:
            output['data']['type'] = str(model.predict([row['content']])[0])
        # Returning exception if an error occurs
        except Exception as ex:
            output['errors'] = [{'message': str(ex)}]
        output_list.append(output)

    return {'values': output_list}    

### 5.0 Create Environment File
Next, create an environment file, called myenv.yml, that specifies all of the script's package dependencies. This file is used to ensure that all of those dependencies are installed in the Docker image. This model needs scikit-learn, pandas, and azureml-sdk.

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")
myenv.add_conda_package("pandas")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [ ]:
with open("myenv.yml","r") as f:
    print(f.read())

### 6.0 Create Configuration File
Create a deployment configuration file and specify the number of CPUs and gigabyte of RAM needed for your ACI container. While it depends on your model, the default of 1 core and 1 gigabyte of RAM is usually sufficient for many models. If you feel you need more later, you would have to recreate the image and redeploy the service.

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "20newsgroups",  "method" : "document classification"}, 
                                               description='Perform document classification against 20newsgroups data')

### 7.0 Deploy to ACI
Estimated time to complete: about 7-8 minutes

Configure the image and deploy. The following code goes through these steps:
1. Build an image using:
    - The scoring file (score.py)
    - The environment file (myenv.yml)
    - The model file
2. Register that image under the workspace. 
3. Send the image to the ACI container.
4. Start up a container in ACI using the image.
5. Get the web service HTTP endpoint.

In [ ]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

# configure the image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml")

service = Webservice.deploy_from_model(workspace=ws,
                                       name='sklearn-newsgroup-classifier',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

In [ ]:
print(service.scoring_uri)

### 8.0 Test Deployed Service

#### 8.1 Import 20newsgroups Test Dataset

In [ ]:
from sklearn.datasets import fetch_20newsgroups

categories = ['comp.graphics', 'sci.space']
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)

X_test = newsgroups_test.data
y_test = newsgroups_test.target

#### 8.2 Format Data in Correct Structure for Cognitive Search
For more information on custom skills see this [link](https://docs.microsoft.com/en-us/azure/search/cognitive-search-custom-skill-interface).

In [ ]:
# send a random row from the test set to score
random_index = np.random.randint(0, len(X_test)-1)

input_data = {"recordId": "0", "data": {"content": newsgroups_test.data[random_index]}}
print(input_data)

#### 8.3 Send HTTP Request and View Results

In [ ]:
import requests
import json

input_json = json.dumps([input_data])

headers = {'Content-Type':'application/json'}

# for AKS deployment you'd need to the service key in the header as well
# api_key = service.get_key()
# headers = {'Content-Type':'application/json',  'Authorization':('Bearer '+ api_key)} 

resp = requests.post(service.scoring_uri, input_json, headers=headers)

print("POST to url", service.scoring_uri)
#print("input data:", input_data)
print("label:", y_test[random_index])
print("prediction:", resp.text)